**Index**

[Preliminary Data Stuff](#data_stuff) <br>
[Data Plan](#assumptions) <br>


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())
%matplotlib inline
from datetime import datetime 

In [8]:
## Team Data
team_data = pd.read_csv('google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MDataFiles_Stage1/MTeams.csv')
team_data.sample(10)


,TeamID,TeamName,FirstD1Season,LastD1Season
336,1437,Villanova,1985,2020
312,1413,UC Davis,2005,2020
131,1232,Indiana St,1985,2020
151,1252,Lipscomb,2003,2020
11,1112,Arizona,1985,2020
259,1360,San Diego,1985,2020
273,1374,SMU,1985,2020
294,1395,TCU,1985,2020
167,1268,Maryland,1985,2020
274,1375,South Alabama,1985,2020


In [9]:
## Massey Data - his file lists out rankings (e.g. #1, #2, #3, ..., #N) 
# of teams going back to the 2002-2003 season, 
# under a large number of different ranking system methodologies. 
massey_data = pd.read_csv('google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MDataFiles_Stage1/MMasseyOrdinals.csv')
massey_data.sample(10)

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank
2854866,2016,128,MCL,1421,108
1892563,2013,92,MB,1162,234
2049810,2014,37,DOK,1275,265
866718,2009,43,ISR,1257,47
1504331,2012,35,MAS,1191,216
2745590,2016,93,CNG,1303,123
3044401,2017,93,DC2,1404,198
1167178,2010,99,DOK,1383,306
3088272,2017,107,DCI,1186,176
3432856,2018,121,ESR,1383,289


In [71]:
## Seed Data - This file identifies the seeds for all teams in each NCAA® tournament, 
# for all seasons of historical data. Thus, there are between 64-68 rows for each year,
# depending on whether there were any play-in games and how many there were.
seed_data = pd.read_csv('google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MDataFiles_Stage1/MNCAATourneySeeds.csv')
seed_data.sample(10)

,Season,Seed,TeamID
1319,2005,Y04,1393
1141,2002,Z04,1326
2128,2017,Y13,1436
2054,2016,Y07,1234
1569,2009,X10,1276
993,2000,Y02,1235
2035,2016,X04,1235
609,1994,Y02,1269
912,1999,X01,1277
1261,2004,Y11,1350


In [75]:

## number of times seeded in the last 5 years  (avg)
## last year seed
## max seed last 5 years

seed_data['Seedless'] = seed_data['Seed'].apply(lambda x: x[1:3])
seed_data['Seedless'] = pd.to_numeric(seed_data['Seedless'])
seed_data['Last Seed'] =  seed_data.groupby('TeamID')['Seedless'].shift(1)
seed_data['Max Seed 5Y'] = seed_data.groupby('TeamID')['Seedless'].transform(lambda x: x.rolling(5, min_periods=1).max())
seed_data['Avg Seed 5Y'] = seed_data.groupby('TeamID')['Seedless'].transform(lambda x: x.rolling(5, min_periods=1).mean())
seed_data['Avg Seed 5Y Weighted'] = seed_data.groupby('TeamID')['Seedless'].transform(lambda x: x.rolling(5, min_periods=1, win_type='triang').mean())
seed_data.drop(['Seedless'], axis=1, inplace=True)

seed_data.loc[seed_data['TeamID'] == 1437]


,Season,Seed,TeamID,Last Seed,Max Seed 5Y,Avg Seed 5Y,Avg Seed 5Y Weighted
55,1985,Z08,1437,NaN,8,8.0,8.000000
105,1986,Y10,1437,8.0,10,9.0,8.666667
229,1988,Y06,1437,10.0,10,8.0,8.333333
363,1990,Y12,1437,6.0,12,9.0,8.750000
392,1991,W09,1437,12.0,12,9.0,8.777778
642,1995,W03,1437,9.0,12,8.0,8.777778
722,1996,X03,1437,3.0,12,6.6,7.333333
771,1997,W04,1437,3.0,12,6.2,5.444444
919,1999,X08,1437,4.0,9,5.4,4.444444
1336,2005,Z05,1437,8.0,8,4.6,4.666667



One row per team per season
| Season | Team_ID |

- Seed Data
- Rank data
- Regular season data 
- Previous Tournament Data
    - Wins previous tournaments, placed last tournament,\

Validation Table
One row per matchup per season - NCAA
| Season | Team_ID_Low | Team_ID_High | Outcome (1,0)|

In [ ]:
## seed data


<a id='assumptions'></a>
#### Data Plan


In [ ]:
### Prediction Data set 
## T1 T2 input_variables 
## T1 T2 output_probability
## 1 - if the team with the lower team # wins
## 0 - if the team with the lower team # loses
## output_probability = prob(1) = probability that the team with lower team number wins
### Lowerteamid_Higherteamid_yr: P(Lowerteamid wins)